## All flights or you can filter if you know airline callsigns...
EJA is netjets
FDY is Southern Airways Express
etc

In [2]:

from opensky_api import OpenSkyApi, FlightTrack
import os.path
import pandas as pd
import folium
import time
import os 
from dotenv import load_dotenv
from folium.plugins import MarkerCluster
load_dotenv()

opensky_user = os.getenv('OPENSKY_USER')
opensky_api_key = os.getenv('OPENSKY_API_KEY')

# Initialize the API (can also be authenticated) 
# dont be dumb if you have the keys
api = OpenSkyApi(opensky_user, opensky_api_key)

def get_fdy():
    # Fetch the current states of all aircraft
    states = api.get_states()

    # Extract data from each StateVector and store in a list
    data = []
    for s in states.states:
        data.append({
            "ICAO24": s.icao24,
            "Callsign": s.callsign,
            "Origin Country": s.origin_country,
            "Latitude": s.latitude,
            "Longitude": s.longitude,
            "Baro Altitude": s.baro_altitude,
            "Geo Altitude": s.geo_altitude,
            "Velocity": s.velocity,
            "True Track": s.true_track,
            "Vertical Rate": s.vertical_rate,
            "On Ground": s.on_ground,
            "Category": s.category,
            "Last Contact": s.last_contact,
            "Position Source": s.position_source,
            "Sensors": s.sensors,
            "SPI": s.spi,
            "Squawk": s.squawk,
            "Time Position": s.time_position
        })

    # Convert list to DataFrame only if FDY is at the start of the callsign. 
    # filter for FDY in the callsign
    df = pd.DataFrame(data)
    df = df[df['Callsign'].str.startswith('FDY')]
    print(f'Flights Found: {len(df)}')
    print(f"Flights on the ground: {len(df[df['On Ground'] == True])}")
    print(f"Flights in the air: {len(df[df['On Ground'] == False])}")
    # Display the first few rows
    # print(df.head())

    # create a csv of to store the icao24 and callsign for each flight. 
    # This will be useful later when we want to filter out flights,
    # or find specific icao codes for tracking
    # if the icao24_codes.csv file exists, load it and only add new icao24 that we don't have:

    if os.path.isfile('icao24_codes.csv'):
        icao24_codes = pd.read_csv('icao24_codes.csv')
        new_data = df[['ICAO24', 'Callsign']].drop_duplicates()
        new_data = new_data[~new_data['ICAO24'].isin(icao24_codes['ICAO24'])]
        icao24_codes = pd.concat([icao24_codes, new_data]).reset_index(drop=True)
        icao24_codes_list = icao24_codes['ICAO24'].tolist()
    else:
        icao24_codes = df[['ICAO24', 'Callsign']].drop_duplicates().reset_index(drop=True)
        
    icao24_codes.to_csv('icao24_codes.csv', index=False)
    # print(icao24_codes_list)
    # return the df and the list of icao24 codes
    return df, icao24_codes, icao24_codes_list

def add_track_to_map(m, track):
    # Check if the track has a path
    if not hasattr(track, 'path') or track.path is None:
        return

    # Get the coordinates of the track path
    coords = [(point[1], point[2]) for point in track.path]

    # Add a line to the map using the coordinates of the track path
    folium.PolyLine(coords, color='blue').add_to(m)

    # Add the callsign to the map over the most recent coordinate
    folium.Marker([coords[-1][0], coords[-1][1]], popup=track.callsign).add_to(m)

# Create a map centered on a default location
m = folium.Map(location=[40, -100], zoom_start=4)  # You can adjust the center and zoom level as needed

# Run the code
df, icao24_codes, icao24_codes_list = get_fdy()
print(df.head(), icao24_codes_list)
for icao24 in icao24_codes_list:
    track = api.get_track_by_aircraft(icao24)
    if track is not None:
        add_track_to_map(m, track)

# Display the map
m


Flights Found: 9
Flights on the ground: 0
Flights in the air: 9
      ICAO24  Callsign Origin Country  Latitude  Longitude  Baro Altitude  \
30    ac231d  FDY493    United States   34.6687   -86.9768        1805.94   
2676  a1afde  FDY183    United States   39.1276   -77.5155         853.44   
5691  aaedcb  FDY107    United States   40.3484   -80.0786        1805.94   
5701  abb40e  FDY3073   United States   39.1675  -120.2830        3695.70   
6963  ad2d61  FDY155    United States   40.3775   -80.0348        1798.32   

      Geo Altitude  Velocity  True Track  Vertical Rate  On Ground  Category  \
30         1866.90     89.55      301.13           0.00      False         2   
2676        891.54     71.47      159.35           0.00      False         0   
5691       1859.28     81.81      320.10           0.00      False         3   
5701       3909.06     95.34      199.22           7.48      False         0   
6963       1844.04     83.25      305.95          -0.65      False       